# Fixpoint Quantization and Overflow #
This notebook shows how number can be represented in binary format and how to (re-)quantize signals. This is also shown practically in Python using ``pyfda_fix_lib`` (numpy based).

In [ ]:
import os, sys
import time

module_path = os.path.abspath(os.path.join('..')) # append directory one level up to import path
if module_path not in sys.path:  # ... if it hasn't been appended already
    sys.path.append(module_path)

import dsp_nmigen.pyfda_fix_lib as fx
%matplotlib inline    

import matplotlib.pyplot as plt

plt.style.use('script.mplstyle')
import numpy as np
import scipy.signal as sig

figsize = {"figsize":(13,7)}

## Theory ##

Unsigned integers are 
The next figure shows how signed integer numbers are represented in two's complement format. The MSB is the sign bit which can be interpreted as the negative value $-2^{W-1}$.
<figure>
    <center>
    <img src='img/twos_complement_signed_int.png' alt='Signed integers in twos complement' width='50%'/>
    <figcaption><b>Fig. x:</b> Signed integers in twos complement</figcaption>
    </center>
</figure>

The same is possible for signed fractional values, although it is important to realize that the *binary point only exists in the developer's head*!  All arithmetic stuff can be implemented with "integer thinking" (and some designers just do that). IMHO, thinking in fractional numbers makes it easier to track the real world value and to separate the range and the resolution.
 
<figure>
    <center>
    <img src='img/twos_complement_signed_frac.png' alt='Signed fractional values in twos complement' width='60%'/> 
    <figcaption><b>Fig. x:</b> Signed fractional values in twos complement</figcaption>
   </center>
 </figure>


### Increasing the wordlength

Before adding two fixpoint numbers their binary points need to be aligned, i.e. they need to have the same number of integer and fractional bits. This can be achieved by extending integer and / or fractional part. 

The value of a fixpoint number doesn't change when zeros are appended to the fractional part (for positive and negative numbers) so that part is easy. 

Prepending zeros to the integer part would change the sign of a negative number. Instead, the integer part is extended with copies of the sign bit, also called **sign extension**. VHDL has the function ``SEXT(std_log_vect, int)`` for achieving this.

<img src="img/requant_extension.png" alt="Extnd integer and fractional part" width="40%"/> 

### Reduce the number of fractional bits

The problem of reducing the number of fractional places is well known from the decimal system. Different methods have been developed to achieve this like rounding, truncation, see e.g. https://www.eetimes.com/an-introduction-to-different-rounding-algorithms for more details than you'll ever need.
 
<figure>
    <img src='img/requant_reduce_fractional.png' alt='Reduce fractional word length' width='80%'/>
    <figcaption>Fig. x: Reduction of fractional wordlength</figcaption>
</figure>

## pyfda_fix_lib

This section uses and describes the fixpoint library `pyfda_fix_lib.py`. A quantizer is constructed as an instance of the class `Fixed()`, its properties are configured with a quantization dict `Q = fx.Fixed(q_dict)`. The quantization dict (in this case `q_dict`) sets the format `QI.QF` at the output of the quantizer and its quantization and overflow behaviour with the following keys:

- **'WI'**   : number of integer bits (integer)
- **'WF'**   : number of fractional bits (integer)
- **'quant'**: requantization behaviour (**'floor'**, 'round', 'fix', 'ceil', 'rint', 'none')
- **'ovfl'** : overflow behaviour (**'wrap'**, 'sat', 'none')
- **'frmt'** : number base / format for output (**'float'**, 'dec', 'bin', 'hex', 'csd'). Non-floats are scaled with 'scale'
- **'scale'**: float or a keyword; the factor between the fixpoint integer representation (FXP) and the real world value (RWV), RWV = FXP / scale. By default, scale = 1 << WI.  If ``scale`` is a float, this value is used. 

     **Examples:**
    
     ```
     WI.WF = 3.0, FXP = "b0110." = 6,   scale = 8 -> RWV = 6 / 8   = 0.75
     WI.WF = 1.2, FXP = "b01.10" = 1.5, scale = 2 -> RWV = 1.5 / 2 = 0.75
     ```
     
     Alternatively, if:

     - ``q_obj['scale'] == 'int'``:   `scale = 1 << self.WF`

     - ``q_obj['scale'] == 'norm'``:  `scale = 2.**(-self.WI)`      


`?fx.Fixed` shows the available options.

### Example
In the following example a quantizer is defined with an output format of 0 integer bits and 3 fractional bits, overflows are wrapped around in two's complement style and additional fractional bits are simply truncated ("floor").

In [ ]:
q_dict = {'WI':0, 'WF': 3,                 # number of integer and fractional bits
          'quant':'floor', 'ovfl': 'wrap'} # quantization and overflow behaviour
Q = fx.Fixed(q_dict)                       # instance of fixpoint class Fixed()
for i in np.arange(12)/10:                 # i = 0, 0.1, 0.2, ...
    print("q<{0:>3.2f}> = {1:>5.3f}".format(i, Q.fixp(i))) # quantize i
# uncomment to show documentation
# ?fx.Fixed 

### Signal Quantization
A sine signal $s(t)$ is quantized in the code below. The plot shows $s(t)$, the quantized signal $s_Q(t)$ the difference between both signals, the quantization error $\epsilon(t)$.

In [ ]:
N = 10000; f_a = 1
t = np.linspace(0, 1, N, endpoint=False)
s = 1.1 * np.sin(2 * np.pi * f_a * t)
#
q_dict = {'WI':0, 'WF': 4, 'quant':'fix', 'ovfl': 'wrap'} # also try 'round' ; 'sat'
Q = fx.Fixed(q_dict) # quantizer instance with parameters defined above

t_cpu = time.perf_counter()

sq = Q.fixp(s) # quantize s
print('Overflows:\t{0}'.format(Q.N_over))
print('Run time:\t{0:.3g} ms for {1} quantizations\n'.format((time.perf_counter()-t_cpu)*1000, Q.N))
#
fig1, ax1 = plt.subplots(**figsize)
ax1.set_title('Quantized Signal $s_Q$({0}.{1}) with Quantizer Settings "{2}", "{3}"'.format(Q.WI, Q.WF, Q.ovfl, Q.quant))
ax1.plot(t, s, label = r'$s(t)$', lw=2)
ax1.step(t, sq, where = 'post', label = r'$s_Q(t)$', lw=2)
ax1.plot(t, s-sq, label = r'$\epsilon(t) = s(t) - s_Q(t)$', lw=2)
ax1.legend(fontsize = 14)
ax1.grid(True)
ax1.set_xlabel(r'$t \rightarrow$'); ax1.set_ylabel(r'$s \rightarrow$');
#

### Transfer Function of the Quantizer
The transfer function of the quantizer shows the quantized signal $s_Q(t)$ over the input signal $s(t)$ (a ramp). The advantage ove reusing the sine from the previous cell is that the input range can be taylored more easily and that the step size is constant.

In [ ]:
Q.resetN()    # reset overflow counter
x = np.linspace(-2, 2, N, endpoint=False) # generate ramp signal
xq = Q.fixp(x) # quantize x
print('Overflows:\t{0}'.format(Q.N_over))
fig2, ax2 = plt.subplots(**figsize); ax2.grid(True)
ax2.set_title('Quantization Transfer Function')
ax2.step(x,xq, where = 'post')
ax2.set_xlabel(r'$x \rightarrow$'); ax2.set_ylabel(r'$x_Q \rightarrow$');

### Number bases and formats
Quantized values can be printed in different number bases, e.g. as a binary string:

In [ ]:
q_dict_f = {'WI':0, 'WF': 8,               # number of ingeger and fractional bits
          'quant':'floor', 'ovfl': 'wrap', # quantization and overflow behaviour
          'frmt':'bin'}                    # output format
Q1 = fx.Fixed(q_dict_f)                    # instance of fixpoint class Fixed()
for i in np.arange(12)/10:                 # i = 0, 0.1, 0.2, ...
    print("q<{0:>3.2f}> = {1}".format(i, Q1.float2frmt(Q1.fixp(i)))) # quantize i + display it in the wanted format

### Format Conversion
Numbers can also be converted back to floating with the method `frmt2float()`:

In [ ]:
Q0 = fx.Fixed({'Q':'3.8', 'quant':'round', 'ovfl':'wrap', 'frmt':'bin'}) # direct setting of quantization options, use 'Q' instead of 
Q1_dict = Q0.q_obj        # read out Q0 quantization dict
Q1_dict.update({'WI':1, 'WF':5})  # update dict
Q1 = fx.Fixed(Q1_dict)    # and create a new quantizer instance with it      
frmt_str = "0.011"
x_org = Q0.frmt2float(frmt_str)
x_q = Q1.frmt2float(frmt_str)
print("q<b{0}> = {1} -> b{2} = {3}".format(frmt_str, x_org, Q1.float2frmt(x_q), x_q))

Bug ?: Illegal characters in the string to be formatted (e.g. frmt2float("0.0x11") do not raise an error but are ignored.

## FIR Filters

The following cell designs an equiripple filter and plots its impulse response (same as the coefficients) and its magnitude frequency response.

In [ ]:
numtaps = 50
b = sig.remez(numtaps,[0,0.1,0.12, 0.5], [1,0], [1,1]) # frequency bands, target amplitude, weights
w, h = sig.freqz(b, [1], worN=2000)
fig, (ax1, ax2) = plt.subplots(2, **figsize); ax1.grid(True); ax2.grid(True)
ax1.set_title('Equiripple Lowpass (FIR) Filter')
ax1.stem(np.arange(numtaps), b, use_line_collection=True)
ax1.set_xlabel(r'$n \rightarrow$'); ax1.set_ylabel(r'$b_n = h[n] \rightarrow$')
ax2.plot(w / (2*np.pi),np.abs(h))
ax2.set_xlabel(r'$F \rightarrow$'); ax2.set_ylabel(r'$|H(F)| \rightarrow$')
fig.set_tight_layout(True)

<figure>
     <center>
    <img src='img/fir_df.png' alt='Direct Form FIR Filter' width='20%'/>
    <figcaption><b>Fig. x:</b> Direct Form FIR Filter</figcaption>
 </center>
</figure>

Fig xx shows the topology of a direct form FIR filter.

## IIR Filters
Unlike FIR filters, IIR filters cannot be implemented with the same elegance and efficiency of array mathematics as each sample depends on the output sample as well. The following code cell designs an elliptic low pass filter and plots its magnitude frequency response.

In [ ]:
b,a = sig.ellip(4,1,40, 2*0.1) # order, pass band ripple, stop band ripple, corner frequency w.r.t. f_S/2
w, h = sig.freqz(b,a, worN=2000)
fig, ax = plt.subplots(1, **figsize); ax.grid(True)
ax.set_title('Elliptic Lowpass (IIR) Filter')
ax.plot(w / (2*np.pi),20*np.log10(np.abs(h)))
ax.set_xlabel(r'$F \rightarrow$'); ax.set_ylabel(r'$|H(F)| \rightarrow$')
print("b=", b)
print("a=", a)

<figure>
     <center>
    <img src='img/iir_df1_df2.png' alt='Direct Form IIR Filter' width='50%'/>
    <figcaption><b>Fig. x:</b> Direct Form FIR Filter Type 1 and 2</figcaption>
 </center>
</figure>

The following two examples show how to implement the most simple recursive filters (no transversal path, i.e. "all-pole filters").

In [ ]:
def IIR1(Q_ACCU, x, a):
    """
    Rekursives Filter mit y[i] = Q< x[i-1] + a y[i-1] >
    """
    y = np.zeros(len(x))
    for i in range(0,len(x)-1):
        y[i+1] = Q_ACCU.fixp(x[i] + a * y[i])
    return y
    
def IIR2(Q_ACCU, x, a):
    """
    Rekursives Filter mit y[i] = Q< x[i-2] + y[i-1] - a y[i-2] >
    """
    y = np.zeros(len(x))
    for i in range(len(x)-2):
        y[i+2] = Q_ACCU.fixp(x[i] + y[i+1] - y[i]*a)
    return y

In [ ]:
alpha = 0.93 # coefficient
N_sim = 200 # number of simulation steps
x = np.zeros(N_sim); x[0] = 1.0 # x is dirac pulse with weight 1
 
q_x = {'WI':0,'WF':3,'quant':'round','ovfl':'sat'} # fixpoint quantization for stimulus
q_coeff = {'WI':0,'WF':3,'quant':'round','ovfl':'wrap'} # coefficient quantization dict
# quantizer settings for accumulator
#q_accu = {'WI':0,'WF':4,'quant':'fix','ovfl':'sat'} # saturation and round towards zero -> no limit cycles
q_accu = {'Q':0.8,'quant':'floor','ovfl':'wrap'}  # große Grenzzyklen bei QI = 0
# kleine Grenzzyklen mit round / floor, abhängig von alpha:
# q_accu = {'WI':0,'WF':4,'quant':'floor','ovfl':'wrap'} 

# Keine Quantisierung ->  Werte für I, F beliebig
q_ideal = {'WI':0,'WF':0,'quant':'none','ovfl':'none'}

Q_coeff = fx.Fixed(q_coeff) # Fixpoint Object mit Parametern "q_coeff"

Q_ideal = fx.Fixed(q_ideal) # Fixpoint-Objekt ohne Quantisierung und Overflow 
Q_accu = fx.Fixed(q_accu) # Fixpoint-Objekt mit Parametern "q_accu"

n = np.arange(N_sim)
t1 = time.perf_counter()
alpha_q = Q_ideal.fixp(alpha)

y = IIR1(Q_ideal, x, alpha_q) # ohne Quantisierung
#yq = IIR2(fx_IIR, x, alpha_q)
yq = IIR1(Q_accu, x, alpha_q)



***
### Copyright

(c) 2016 - 2020 Christian Münker 

This notebook is provided as [Open Educational Resource](https://en.wikipedia.org/wiki/Open_educational_resources) , feel free to use it for your own purposes. Please attribute the work as follows: *Christian Münker, dsp_migen documentation*.